In [2]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt

def compute_camera_calibration_matrix_and_undistort(image):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    images = glob.glob('camera_cal/calibration*.jpg')
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
    
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, image.shape[1:], None, None)
    return cv2.undistort(image, mtx, dist, None, mtx)

def perspective_transform(calibrated):
    img_size = (calibrated.shape[1],  calibrated.shape[0])
    src = np.float32([[280, 680], [1110, 680], [520, 505], [800, 505]])
    dst = np.float32([[280, img_size[1]], [1110, img_size[1]], [280, 0],[1110, 0]])
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(calibrated, M, img_size)
    Minv = cv2.getPerspectiveTransform(dst, src)
    return warped, Minv

def thresholded_binary(transformed, s_thresh=(170, 255), sx_thresh=(20, 100)):
    hls = cv2.cvtColor(transformed, cv2.COLOR_BGR2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1    
    # Threshold color chann                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               bbnnnbel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    combined = np.zeros_like(sxbinary)
    combined[(sxbinary == 1) | (s_binary == 1)] = 1
    return combined

def find_lane_pixels(binary_warped):
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    nwindows = 9
    margin = 50
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                         (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty

def search_around_existing_poly(binary_warped):
    margin = 50

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    left_prime_area = left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2]
    left_lane_inds = (nonzerox > left_prime_area - margin) & (nonzerox < left_prime_area + margin)
    right_prime_area = right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2]
    right_lane_inds = (nonzerox > right_prime_area - margin) & (nonzerox < right_prime_area + margin)
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty

def fit_new_poly(leftx, lefty, rightx, righty, ploty):
    left_fit = np.polyfit(lefty, leftx, 2) 
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        last_known_good_left_fit = left_fit
    except TypeError:
        left_fit = None
        left_fitx = last_known_good_left_fit[0]*ploty**2 + last_known_good_left_fit[1]*ploty +  last_known_good_left_fit[2]

    right_fit = np.polyfit(righty, rightx, 2)
    try:
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        last_known_good_right_fit = right_fit
    except TypeError:
        right_fit = None
        right_fitx = last_known_good_right_fit[0]*ploty**2 + last_known_good_right_fit[1]*ploty + last_known_good_right_fit[2]

    # offset calculation
    y_eval = np.max(ploty)
    
    bottom_left_x = left_fit[0]*y_eval**2 + left_fit[1]*y_eval + left_fit[2] if left_fit is not None else 0
    bottom_right_x = right_fit[0]*y_eval**2 + right_fit[1]*y_eval + right_fit[2] if right_fit is not None else 0
    center = (bottom_left_x + bottom_right_x) / 2
    offset = (1280/2 - center) * (3.7/700)

    return left_fitx, right_fitx, ploty, offset

# Polynomial fit values from the previous frame
left_fit = None
right_fit = None
last_known_good_left_fit = None
last_known_good_left_fit = None
def find_lane_lines(binary_warped):
    # sanity check
    if left_fit is not None and right_fit is not None:
        leftx, lefty, rightx, righty = search_around_existing_poly(binary_warped)
    else:
        leftx, lefty, rightx, righty = find_lane_pixels(binary_warped)

    ploty = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0])
    left_fitx, right_fitx, ploty, offset = fit_new_poly(leftx, lefty, rightx, righty, ploty)
    avg_curvature = measure_curvature_real(leftx, lefty, rightx, righty, ploty)
    
    return  left_fitx, right_fitx, avg_curvature, ploty, offset

def measure_curvature(fit_coeffs, y_eval):
    if fit_coeffs is not None and fit_coeffs[0] != 0:
        return ((1 + (2 * fit_coeffs[0] * y_eval + fit_coeffs[1]) ** 2) ** (3/2)) / abs(2 * fit_coeffs[0])
    else:
        print("invalid")
        raise Exception("invalid")

last_known_good_left_curverad = 0
last_known_good_right_curverad = 0
def measure_curvature_real(leftx, lefty, rightx, righty, ploty):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30.0/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    y_eval = np.max(ploty) * ym_per_pix
    left_fit_cr = np.polyfit(lefty * ym_per_pix, leftx * xm_per_pix, 2) 
    try:
        left_curverad = measure_curvature(left_fit_cr, y_eval)
        last_known_good_left_curverad = left_curverad
    except:
        left_curverad = last_known_good_left_curverad
    
    right_fit_cr = np.polyfit(righty * ym_per_pix, rightx * xm_per_pix, 2) 
    try:    
        right_curverad = measure_curvature(right_fit_cr, y_eval)
        last_known_good_right_curverad = right_curverad
    except:
        right_curverad = last_known_good_right_curverad

    return (left_curverad + right_curverad) / 2

def project_lane_detection_back_to_image(thresholded_binary_image, left_fitx, right_fitx, ploty, Minv, calibrated):
    shape = thresholded_binary_image.shape
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(thresholded_binary_image).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (shape[1], shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(calibrated, 1, newwarp, 0.3, 0)
    return result

def process_image(image):
    try:
        calibrated = compute_camera_calibration_matrix_and_undistort(image)
        # plt.imshow(calibrated)
        # plt.show()
        transformed, Minv = perspective_transform(calibrated)
        # plt.imshow(transformed)
        # plt.show()
        thresholded_binary_image = thresholded_binary(transformed)
        # plt.imshow(thresholded_binary_image)
        # plt.show()
        left_fitx, right_fitx, avg_curvature, ploty, offset = find_lane_lines(thresholded_binary_image)
        result = project_lane_detection_back_to_image(thresholded_binary_image, left_fitx, right_fitx, ploty, Minv, calibrated)
        cv2.putText(result, f"radius of curvature = {avg_curvature} m", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        cv2.putText(result, f"offset = {offset} m", (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        #plt.imshow(result)
        #plt.show()
        return result
    except Exception as e:
        print(e)

In [7]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

output = 'project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4").subclip(5, 8)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(output, audio=False)
'''
img = cv2.imread('test_images/test1.jpg')
result = process_image(img)
plt.imshow(result)
plt.show()
'''

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


 99%|█████████▊| 75/76 [05:22<00:04,  4.33s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

CPU times: user 5min 15s, sys: 2.21 s, total: 5min 17s
Wall time: 5min 25s


"\nimg = cv2.imread('test_images/test1.jpg')\nresult = process_image(img)\nplt.imshow(result)\nplt.show()\n"